In [2]:
%pip install --only-binary=:all: tiktoken
%pip install --upgrade pip setuptools wheel
%pip install tiktoken --only-binary=:all:

  Using cached tiktoken-0.11.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tiktoken-0.11.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.
  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 25.2
    Uninstalling pip-25.2:
      Successfully uninstalled pip-25.2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Install required libraries
%pip install -qU \
    langchain==0.3.* \
    langchain_openai==0.3.* \
    langchain_community \
    unstructured[md]==0.17.* \
    langgraph==0.4.* \
    websockets==15.0.*

Note: you may need to restart the kernel to use updated packages.


In [1]:
import langchain

from langchain_community.utilities.requests import RequestsWrapper
from langchain_community.agent_toolkits.openapi import planner
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec
import requests

from langchain_community.document_loaders.text import TextLoader
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableMap, RunnableAssign
from langchain_core.documents.base import Document

from dotenv import load_dotenv
from pprint import pp
from IPython.display import Markdown as render_md
import pynvml  # type: ignore[import]
from dotenv import load_dotenv

import os

# openai api need to be set up before runnung the below cell. 

import getpass # for testing. remove for compilation 

os.environ['OPENAI_API_KEY'] = getpass.getpass("Enter your OpenAI API key: ")

def get_supporting_documents():
    current = os.getcwd()

    while True:
        candidate = os.path.join(current, 'GAI-3101-CAP')
        if os.path.isdir(candidate):
            target = os.path.join(candidate, 'capstone/support-info')
            return os.path.relpath(target, os.getcwd())

        parent = os.path.dirname(current)
        if parent == current:
            break  # Reached root without finding 'GAI-3101-CAP'
        current = parent

    raise FileNotFoundError("Could not find a directory containing 'GAI-3101-CAP'")

SUPPORT_DIR = get_supporting_documents()


def get_files_with_extensions(folder_path, extensions):
    matched_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if any(file.lower().endswith(ext.lower()) for ext in extensions):
                matched_files.append(os.path.join(root, file))
    return matched_files

get_files_with_extensions(SUPPORT_DIR, ['.md'])

md_paths = get_files_with_extensions(SUPPORT_DIR,['.md'])
md_docs = []
for path in md_paths:
    text_loader = TextLoader(path)
    doc = text_loader.load()
    md_docs += doc



embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
)
vector_store = InMemoryVectorStore(embedding_model)
vector_store.add_documents(documents=md_docs)


doc_retriever = vector_store.as_retriever(
    search_type="similarity", #could also do MMR if we want to avoid repeated/very similar docs
    search_kwargs={
        'k': 3 # number of documents to consider
    }
)

system_msg = '''
You are a helpful AI bot being used to assist Beanbotics., a company that sell automated robotic arms that make coffee.
Your job is to automate the initial processing of
support tickets. You will read incoming tickets, extract key information,
classify the requests into categories, assign priority levels, and update the ticketing
system with this information

Instructions:
- Use only the content found in the provided context documents unless explicitly asked to use outside information
- Include inline citations using the `source` field from metadata (e.g., [source]).
- If documents conflict, describe the differing perspectives and cite each source (e.g., [source A], [source B]).
- If the answer cannot be found, say: “The context does not contain that information.”
- If the question is ambiguous, explain your interpretation before answering.'
'''

human_msg = """
Please answer this query from the user
<original user query>
{query}
</original user query>

---
Use these docs to answer the original user query
<context documents>
{context_docs}
</context documents>
"""

generation_template = ChatPromptTemplate([
    ("system", system_msg),
    ("human", human_msg),
])

# placing the chain here so the RAG system can influence instructions 
rag_chain = (
    RunnableMap(
        query=RunnablePassthrough(),
        context_docs=doc_retriever
    )
    | RunnableMap(
        context_docs=RunnableLambda(lambda x: x['context_docs']),
        llm_response=(
            generation_template
            | llm
            | StrOutputParser()
            | render_output
        )
    )
)

rag_result = rag_chain.invoke("What is the bean machine")

print(rag_result)

#enriched_instructions = f"""
#Use the following documentation context to complete the task:

#{rag_result['llm_response']}

#Original instructions:
#{instructions}
#"""


/tmp/ipykernel_29652/621055188.py:22: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Enter your OpenAI API key:  ········


NameError: name 'llm' is not defined

In [27]:
# Load variables from .env file
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
if (openai_api_key):
    print("------------------- Loaded Key -------------------")
# print(openai_api_key)  # optional: check it loaded correctly

# Load the OpenAPI specification from the running ticketing system
root = "http://localhost:3000"
api_spec_url = f"{root}/api/docs/openapi.json"

# Download and parse the OpenAPI spec
response = requests.get(api_spec_url)
data = response.json()
data['servers'] = [{'url': root}]
openapi_spec = reduce_openapi_spec(data, dereference=False)

### Try to get it to categorize all tickets
requests_wrapper = RequestsWrapper()
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.0)

import requests

agent = planner.create_openapi_agent(
    api_spec=openapi_spec,
    requests_wrapper=requests_wrapper,
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True,
    handle_parsing_errors=True,
    allow_operations=['GET', 'POST', 'PUT', 'PATCH', 'DELETE']
)


response = requests.get(root+ "/api/tickets")
tickets = response.json()

for t in tickets:
    ticket_id = t["id"]
    ticket_cat = t["category"]
    print("Ticket:", ticket_id)

import json
import websockets

WS_URL = "ws://localhost:3000/ws"
# This async function connects to the WebSocket and listens for ticket updates
# Once a ticket update is received, it yields it for processing.
async def listen_for_ticket_updates():
    print("\n------------------- Starting Connection and Agent -------------------")
    # Establish a connection to the WebSocket server
    async with websockets.connect(WS_URL) as websocket:
        print("WebSocket connection established.")
        try:
            # Keep listening for messages from the server
            while True:
                message = await websocket.recv()  # Wait for a new message
                yield json.loads(message)
        except websockets.ConnectionClosed:
            print("WebSocket connection closed.")
        except Exception as e:
            print(f"WebSocket error: {e}")




# --- Async wrapper for the agent ---
async def run_agent_async(instructions: str):
    rag_result = rag_chain.invoke({"query": instructions})
    """Run the agent in a non-blocking way."""
    enriched_instructions = f"""
Use the following documentation context to complete the task:

{rag_result['llm_response']}

Original instructions:
{instructions}
"""
    # agent.invoke is synchronous, so run it in a thread
    return await asyncio.to_thread(agent.invoke, enriched_instructions)

# --- Async agent helper ---
async def run_agent(ticket_id, action_description, instructions):
    """Run an agent action asynchronously with logging."""
    print(f'{action_description} for ticket: {ticket_id}')
    return await run_agent_async(instructions.strip())


# --- Simplified async ticket functions ---
async def determine_category(ticket_id):
    await run_agent(
        ticket_id,
        "Categorizing ticket",
        f"""
Assign this ticket {ticket_id} one category from: Mechanical, Quality, Maintenance, Technical, Awaiting Details.
Use “Awaiting Details” ONLY if insufficient info exists. Otherwise, POST the assigned category.
"""
    )

async def determine_priority(ticket_id):
    await run_agent(
        ticket_id,
        "Determining ticket priority",
        f"""
Assign this ticket {ticket_id} a priority: High, Medium, or Low. 
If the category is “Awaiting Details,” set priority to Low. Otherwise, POST the assigned priority.
"""
    )

async def determine_response(ticket_id):
    await run_agent(
        ticket_id,
        "Checking for automatic response",
        f"""
GET ticket {ticket_id}. 
If category is Awaiting Details, POST a response requesting more info; otherwise do nothing.
Use JSON: ticket_id={ticket_id}, author="Support Agent", message="<friendly message>"
"""
    )

async def determine_status(ticket_id):
    await run_agent(
        ticket_id,
        "Checking ticket status",
        f"""
Check ticket {ticket_id}. 
If priority is High and ticket seems urgent, update status to "escalated". Otherwise, do nothing.
"""
    )

async def determine_escalation(ticket_id):
    await run_agent(
        ticket_id,
        "Checking if escalation is required",
        f"""
Check ticket {ticket_id}. 
If status is escalated, POST an automatic escalation response: 
ticket_id={ticket_id}, author="Support Agent", message="<context-aware escalation message>"
"""
    )

async def auto_respond(ticket_id):
    await run_agent(
        ticket_id,
        "Checking if auto-response is needed",
        f"""
GET ticket {ticket_id}'s most recent response. 
If last response is from Support Agent, ignore. Otherwise, POST a placeholder response using the provided documentation:
ticket_id={ticket_id}, author="Support Agent", message="<friendly placeholder>"
"""
    )


# --- Async listener loop ---
async for message in listen_for_ticket_updates():
    ticket_id = message.get('ticketId')
    update_type = message.get('updateType')
    print("----------------- MESSAGE -------------")
    print(message)
    print("----------------- Ticket ID -------------")
    print(ticket_id)
    print("----------------- UPDATE TYPE -------------")
    print(update_type)

    if update_type == 'created':
        print("CREATED")
        # Run all ticket handlers asynchronously in sequence
        await determine_category(ticket_id)
        await determine_priority(ticket_id)
        await determine_response(ticket_id)
        await determine_status(ticket_id)
        await determine_escalation(ticket_id)

    elif update_type == 'response':
        await auto_respond(ticket_id)

------------------- Loaded Key -------------------


ConnectionError: HTTPConnectionPool(host='localhost', port=3000): Max retries exceeded with url: /api/docs/openapi.json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fdea3309690>: Failed to establish a new connection: [Errno 111] Connection refused'))